In [ ]:
import sys
from pathlib import Path

# Go two levels up to reach project root
project_root = Path.cwd().parents[1]  # from subfolder/ to notebook_folder/ to project_root/
scripts_path = project_root / "scripts"

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

In [ ]:
from scripts.general_scripts.features_engineering import quicky_data, lag_features_indicators
from scripts.general_scripts.pipelines import change_model
from scripts.general_scripts.predict import future_change_prediction
from scripts.general_scripts.lstm import LSTMModelMultiStep, LSTMModelMultiOutput
from scripts.general_scripts.helper import CustomizedLoss
import pandas as pd
import torch.nn as nn 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
scaler = StandardScaler()
model_type = LSTMModelMultiOutput
criterion = nn.MSELoss()
n_lags = 25
n_forecast = 1

In [ ]:
# 📂 Load dataset
file_path_1 = "../ready_data/change_vn_index.csv"
df_1 = pd.read_csv(file_path_1)
df_1 = quicky_data(df_1)

In [ ]:
df_1.tail(20)

In [ ]:
for i in range(5):
    baseline = ExponentialSmoothing(
    df_1['VN_Index_Close'].iloc[:-n_forecast*(i+1)],
    trend='add',            # additive trend
    seasonal='add',         # additive seasonality
    seasonal_periods=261    # one business‐year ≈ 261 days
    ).fit()

    # 2) Forecast the next day (one‐step ahead):
    price_forecast = baseline.forecast(n_forecast)

    # 3) If you want to extract the trend/seasonal forecasts separately:
    fitted_components = baseline.fittedvalues  # this is price = level+trend+seasonal
    level = baseline.level                     # the “smoothed level” ≈ trend
    seasonal = baseline.season                 # the seasonal factors

    print(price_forecast)

    data = df_1[['Change', 'VN_Index_Close']]

    if i:
        data = data.iloc[:-n_forecast*i]
    # 🚀 Train the model and get the test set
    model, X_test_tensor, scaler, y_pred = change_model(data, df_1, scaler, model_type, criterion, tuning=False, train_seq_len=n_lags, test_seq_len=n_forecast, epochs=50)

In [ ]:
# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = change_model(data, df_1, scaler, model_type, criterion, tuning=False, train_seq_len=n_lags, test_seq_len=n_forecast, epochs=50)

# 🔮 Generate future predictions
future_change_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)

In [ ]:
# 📊 Select additional VN-INDEX statistics for prediction
data = df_1

# 🚀 Train the model and get the test set
model, X_test_tensor, scaler, y_pred = change_model(data, df_1, scaler, model_type, criterion)

# 🔮 Generate future predictions
future_change_prediction(X_test_tensor, data, y_pred, scaler, model, num_days=n_forecast)